In [0]:
%config IPCompleter.greedy=True

!git clone https://github.com/MultiAgentLearning/playground ~/playground
%cd ~/playground
!pip install -U .

!git clone https://github.com/JaroslavUrbann/Pommerman.git /content/Pommerman
%cd /content/Pommerman

!pip3 install -r requirements.txt

In [0]:
# Class that creates and displays a video
%cd /content/Pommerman
import cv2
import numpy as np
from PIL import Image
import os
import base64
from IPython.display import HTML
import time

class VideoMaker:
    _coded_frames = []
    _sprite_folder = "/content/Pommerman/CustomSprites"
    _mp4_path = "Videos/vid.mp4"
    _video_path = "Videos/video.mp4"
    _mappings = {
        "231, 76, 60": "GreenBomberman.png",
        "46, 139, 87": "RedBomberman.png",
        "65, 105, 225": "YellowBomberman.png",
        "238, 130, 238": "BlueBomberman.png",
        "240, 248, 255": "BackgroundTile.png",
        "128, 128, 128": "SolidBlock.png",
        "210, 180, 140": "ExplodableBlock.png",
        "255, 153, 51": "BombBackground.png",
        "241, 196, 15": "FlameBackground.png",
        "141, 137, 124": "fog", # fog
        "153, 153, 255": "BombPowerupBackground.png",
        "153, 204, 204": "FlamePowerupBackground.png",
        "97, 169, 169": "SpeedPowerupBackground.png",
        "48, 117, 117": "agent" # dummy agent
    }

    def add_coded_frame(self, frame):
        self._coded_frames.append(frame)
    
    def _decode_square(self, code):
        key = ', '.join([str(int(n)) for n in code])
        path = os.path.join(self._sprite_folder, self._mappings[key])
        img = Image.open(path).convert('RGB')
        return np.array(img)

    def _decode_frame(self, frame):
        hd_frame = np.zeros((64 * 11, 64 * 11, 3), dtype="uint8")
        for y in range(11):
            for x in range(11):
                square = self._decode_square(frame[y, x])
                hd_frame[y * 64:(y + 1) * 64, x * 64:(x + 1) * 64] = square
        return hd_frame

    def _create_video(self):
        out = cv2.VideoWriter(self._mp4_path, cv2.VideoWriter_fourcc(*'mp4v'), 10, (64 * 11, 64 * 11))
        for i in range(len(self._coded_frames)):
            next_frame = self._decode_frame(self._coded_frames[i])
            out.write(next_frame)
#             print("next frame " + str(i))
        out.release()

    def show_video(self):
        self._create_video()
        os.system("ffmpeg -i " + self._mp4_path + " -vcodec libx264 " + self._video_path + " -y")
        os.system("rm " + self._mp4_path)
        video = open(self._video_path, "rb").read()
        os.system("rm " + self._video_path)
        encoded = base64.b64encode(video)
        return HTML(data='''<video alt="test" autoplay 
                 loop controls style="height: 704px;width: 704px;">
                 <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii')))

In [0]:
from PIL import Image
from IPython import display
import pommerman
from pommerman import agents
import matplotlib.pyplot as plt
%matplotlib inline

from pretraining.large_network import LargeNetwork
from pretraining import pretraining_game_env, pretraining_database
from agents.LN_agent import LNAgent
from testing.testing_game_env import test_network
import gdrive_auth

drive = gdrive_auth.get_drive()
LN = LargeNetwork(drive=drive)
LN.load_model(model_id="", log_id="")
# LN.init_model("")
kwargs1 = {"a_id": 1, "n_id": 1, "LN": LN}
kwargs2 = {"a_id": 2, "n_id": 1, "LN": LN}
agent1 = LNAgent
agent2 = LNAgent

In [0]:
agent_list = [
    agents.SimpleAgent(),
    agent1(**kwargs1),
    agents.SimpleAgent(),
    agent2(**kwargs2),
]

env = pommerman.make('PommeFFA-v1', agent_list)

VidMaker = VideoMaker()

for i_episode in range(3):
    state = env.reset()
    done = False
    while not done:
        frame = env.render(mode='rgb_array')
        VidMaker.add_coded_frame(frame)

        actions = env.act(state)
        state, reward, done, info = env.step(actions)
env.close()
VidMaker.show_video()

In [0]:
from PIL import Image
im = Image.open("/content/Pommerman/CustomSprites/BackgroundTile.png").convert('RGB')
im.show()